In [3]:
from tqdm import tqdm

import numpy as np

import sgkit as sg
from sgkit.io.vcf import vcf_to_zarr

import tskit
import tszip

import sys
sys.path.append("../../tsimpute/src/")
import compare_vcfs as cv
import impute_by_sample_matching as impute
import util

import lshmm


In [4]:
trees_dir = "../data/trees/"
zarr_dir = "../data/zarr/"
vcf_dir = "../data/vcf/"
lshmm_dir = "../analysis/lshmm/"


In [5]:
ref_ts_file = trees_dir + "ref.tsz"
ref_ts = tszip.decompress(ref_ts_file)


In [6]:
ref_ds_compat = sg.load_dataset(zarr_dir + "ref.zarr")
ref_ds_compat


<xarray.Dataset>
Dimensions:               (variants: 504314, samples: 700, ploidy: 2,
                           contigs: 1, filters: 1, alleles: 4)
Dimensions without coordinates: variants, samples, ploidy, contigs, filters,
                                alleles
Data variables: (12/14)
    call_genotype         (variants, samples, ploidy) int8 dask.array<chunksize=(10000, 700, 2), meta=np.ndarray>
    call_genotype_mask    (variants, samples, ploidy) bool dask.array<chunksize=(10000, 700, 2), meta=np.ndarray>
    call_genotype_phased  (variants, samples) bool dask.array<chunksize=(10000, 700), meta=np.ndarray>
    contig_id             (contigs) <U2 dask.array<chunksize=(1,), meta=np.ndarray>
    contig_length         (contigs) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    filter_id             (filters) object dask.array<chunksize=(1,), meta=np.ndarray>
    ...                    ...
    variant_contig        (variants) int8 dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_filter        (variants, filters) bool dask.array<chunksize=(10000, 1), meta=np.ndarray>
    variant_id            (variants) object dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_id_mask       (variants) bool dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_position      (variants) int32 dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_quality       (variants) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
Attributes:
    contig_lengths:        [28050000]
    contigs:               ['20']
    filters:               ['PASS']
    max_alt_alleles_seen:  2
    source:                sgkit-0.7.0
    vcf_header:            ##fileformat=VCFv4.2\n##FILTER=<ID=PASS,Descriptio...
    vcf_zarr_version:      0.2

In [7]:
target_chip_ds_compat = sg.load_dataset(zarr_dir + "target_chip_compat.zarr")
target_chip_ds_compat


<xarray.Dataset>
Dimensions:             (variants: 7899, samples: 176, ploidy: 2, contigs: 1,
                         alleles: 4)
Dimensions without coordinates: variants, samples, ploidy, contigs, alleles
Data variables:
    call_genotype       (variants, samples, ploidy) float64 dask.array<chunksize=(1975, 44, 1), meta=np.ndarray>
    call_genotype_mask  (variants, samples, ploidy) bool dask.array<chunksize=(3950, 88, 1), meta=np.ndarray>
    contig_id           (contigs) <U2 dask.array<chunksize=(1,), meta=np.ndarray>
    sample_id           (samples) object dask.array<chunksize=(176,), meta=np.ndarray>
    variant_allele      (variants, alleles) <U1 dask.array<chunksize=(7899, 4), meta=np.ndarray>
    variant_contig      (variants) int8 dask.array<chunksize=(7899,), meta=np.ndarray>
    variant_position    (variants) int32 dask.array<chunksize=(7899,), meta=np.ndarray>
Attributes:
    contigs:  ['20']
    source:   sgkit-0.7.0

In [8]:
target_test_ds = target_chip_ds_compat.sel(samples=target_chip_ds_compat.sample_id.isin(['HGDP01408']).values)
query_h = impute.prepare_input_matrix(ref_ts, target_test_ds)
query_h.shape


  0%|          | 0/7899 [00:00<?, ?it/s]

100%|██████████| 7899/7899 [00:25<00:00, 311.21it/s]


(2, 504314)

In [9]:
ref_h = ref_ts.genotype_matrix(alleles=tskit.ALLELES_ACGT)
ref_h.shape


(504314, 1400)

In [10]:
n_haps = ref_h.shape[1]
mu = util.get_mismatch_probability(n_haps=n_haps) * np.ones(ref_ts.num_sites)
rho = util.get_switch_probability(n_haps=n_haps, site_positions=ref_ts.sites_position)
print(n_haps)
print(mu)
print(rho)


1400
[4.61130459e-05 4.61130459e-05 4.61130459e-05 ... 4.61130459e-05
 4.61130459e-05 4.61130459e-05]
[0.00000000e+00 3.99920011e-04 5.71265337e-04 ... 2.85710204e-05
 4.28479605e-04 1.71413878e-04]


In [11]:
print(ref_h.shape)
print(query_h.shape)
print(type(ref_h))
print(type(query_h))


(504314, 1400)
(2, 504314)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [12]:
path_duncan, ll_duncan = lshmm.viterbi(
    reference_panel=ref_h,
    query=np.array([query_h[0, :]]),
    recombination_rate=rho,
    mutation_rate=mu[0],
)


In [13]:
_, path_tskit, _ = impute.impute_by_sample_matching(
    ref_ts=ref_ts,
    target_ds=target_test_ds,
    switch_prob=rho,
    mismatch_prob=mu,
    precision=25,
)


100%|██████████| 504314/504314 [00:03<00:00, 150605.47it/s]


In [14]:
path_duncan


array([37, 37, 37, ..., 27, 27, 27])

In [15]:
path_tskit


array([[1291, 1291, 1291, ..., 1006, 1006, 1006],
       [1259, 1259, 1259, ...,  187,  187,  187]], dtype=int32)

In [16]:
print(np.sum(np.invert(np.equal(path_duncan[:-1], path_duncan[1:]))))
print(np.sum(np.invert(np.equal(path_tskit[0][:-1], path_tskit[0][1:]))))


208
208


In [17]:
switch_idx_duncan = np.where(np.invert(np.equal(path_duncan[:-1], path_duncan[1:])))[0]
switch_idx_tskit = np.where(np.invert(np.equal(path_tskit[0][:-1], path_tskit[0][1:])))[0]


In [18]:
switch_idx_duncan


array([  1137,   2932,   7027,   8535,   9799,  10776,  11819,  12022,
        14643,  17149,  17578,  20391,  21805,  22400,  22538,  26975,
        28360,  29849,  37924,  39002,  42292,  43428,  48641,  50621,
        50931,  55763,  65575,  67095,  68243,  69190,  69967,  71266,
        72583,  73453,  75053,  76353,  77765,  78555,  79631,  80099,
        80700,  81998,  82954,  84356,  86113,  87146,  88566,  89210,
        89776,  92426,  96885,  97407,  98491,  99896, 101651, 102471,
       105430, 108623, 109712, 115495, 117647, 124645, 126054, 128518,
       129704, 132372, 133179, 134771, 137473, 140251, 142682, 147283,
       150085, 153058, 154597, 158335, 159717, 161106, 163636, 165461,
       166165, 178771, 184053, 186727, 187809, 189705, 192462, 193671,
       195434, 196723, 197820, 204909, 205723, 206378, 207366, 208882,
       210422, 211512, 213682, 214529, 217977, 221890, 226179, 226549,
       228868, 230091, 232303, 236782, 238865, 240263, 243780, 245332,
      

In [19]:
switch_idx_tskit


array([  1137,   2932,   7027,   8535,   9799,  10776,  11819,  12022,
        14643,  17149,  17578,  20391,  21805,  22400,  22538,  26975,
        28360,  29849,  37924,  39002,  42292,  43428,  48641,  50621,
        50931,  55763,  65575,  67095,  68243,  69190,  69967,  71266,
        72583,  73453,  75053,  76353,  77765,  78555,  79631,  80099,
        80700,  81998,  82954,  84356,  86113,  87146,  88566,  89210,
        89776,  92426,  96885,  97407,  98491,  99896, 101651, 102471,
       105430, 108623, 109712, 115495, 117647, 124645, 126054, 128518,
       129704, 132379, 133179, 134771, 137473, 140251, 142682, 147283,
       150085, 153058, 154597, 158335, 159717, 161106, 163636, 165461,
       166165, 178771, 184053, 186727, 187809, 189705, 192462, 193671,
       195434, 196723, 197820, 204909, 205723, 206378, 207366, 208882,
       210422, 211512, 213682, 214529, 217977, 221890, 226179, 226549,
       228868, 230091, 232303, 236782, 238865, 240263, 243780, 245332,
      

In [20]:
np.where(np.equal(switch_idx_duncan, switch_idx_tskit) == False)


(array([ 65, 136]),)

In [21]:
print(switch_idx_duncan[65])
print(switch_idx_tskit[65])


132372
132379


In [22]:
def check_alleles(alleles, num_sites):
    """
    Checks the specified allele list and returns a list of lists
    of alleles of length num_sites.

    If alleles is a 1D list of strings, assume that this list is used
    for each site and return num_sites copies of this list.

    Otherwise, raise a ValueError if alleles is not a list of length
    num_sites.
    """
    if isinstance(alleles[0], str):
        return [alleles for _ in range(num_sites)]
    if len(alleles) != num_sites:
        raise ValueError("Malformed alleles list")
    return alleles


def ls_path_log_probability(h, path, alleles, G, rho, mu):
    """
    Returns the log probability of the specified path through the genotypes for the
    specified haplotype.
    """
    assert rho[0] == 0
    m, n = G.shape
    alleles = check_alleles(alleles, m)
    # TODO It's not entirely clear why we're starting with a proba of 1 / n for the
    # model. This was done because it made it easier to compare with an existing
    # HMM implementation. Need to figure this one out when writing up.
    log_proba = np.log(1 / n)
    for site in range(0, m):
        if len(alleles[site]) > 1:
            assert mu[site] <= 1 / (len(alleles[site]) - 1)
        pe = mu[site]
        if h[site] == G[site, path[site]] or h[site] == tskit.MISSING_DATA:
            pe = 1 - (len(alleles[site]) - 1) * mu[site]
        assert 0 <= pe <= 1
        pt = rho[site] / n
        if site == 0 or path[site] == path[site - 1]:
            pt = 1 - rho[site] + rho[site] / n
        assert 0 <= pt <= 1
        log_proba += np.log(pt) + np.log(pe)
    return log_proba


In [29]:
ll_tskit = ls_path_log_probability(
    h=query_h[0, :],
    path=path_tskit[0, :],
    alleles=tskit.ALLELES_ACGT,
    G=ref_h,
    rho=rho,
    mu=mu,
)


In [30]:
ll_duncan = ls_path_log_probability(
    h=query_h[0, :],
    path=path_duncan,
    alleles=tskit.ALLELES_ACGT,
    G=ref_h,
    rho=rho,
    mu=mu,
)


In [31]:
print(ll_tskit)
print(ll_duncan)


-3570.375991932556
-3570.375991932556
